In [1]:
import gzip, json, re, requests


In [2]:
with gzip.open('/data/NPMvulnerabilities/NPMPackages.json.gz', 'rb') as f:
    fc = f.read().decode("utf-8")

In [3]:
from json import JSONDecoder, JSONDecodeError
import re

NOT_WHITESPACE = re.compile(r'[^\s]')

def decode_stacked(document, pos=0, decoder=JSONDecoder()):
    while True:
        match = NOT_WHITESPACE.search(document, pos)
        if not match:
            return
        pos = match.start()

        try:
            obj, pos = decoder.raw_decode(document, pos)
        except JSONDecodeError:
            # do something sensible if there's some error
            pass
        yield obj

In [12]:
with open('../new/onlytenKpop.csv','r') as f:
    pkgs = f.read()
pkgs = pkgs.replace('"','')
pkgs = pkgs.split(',')
pkgs[-1] = pkgs[-1].rstrip()

In [20]:
repodict = {}
for obj in decode_stacked(fc):
    objname = obj['name']
    if objname in pkgs:
        d = obj['versions']
        ghome = ''
        for v in d.values():
            try: 
                ghome = v['repository']['url']
                
            except: 
                try:
                    ghome = v['homepage']
                    
                except:
                    try:
                        ghome = v['repository']
                        
                    except:
                        continue
        if ghome == '':
            #outstr += '#Home not found ' + objname +'\n'
            #print ('#Home not found ', objname)
            pass
        else:
            try: 
                ghome2 = ghome.replace('www.','').split('.')[1]
                try:
                    if ghome.replace('www.','').split('.')[2] =='js':
                        ghome2 = ghome2 + '.js'
                except: pass
            except: 
                pass            
            m = re.sub('^com[/:]','',ghome2)
            m = m.split('/tree')[0]
            if '/' not in m:
                try:
                    ghome2 = ghome.replace('www.','').split('.')[2]
                    try:
                        if ghome.replace('www.','').split('.')[2] =='js':
                            ghome2 = ghome2 + '.js'
                    except:
                        pass
                    m = re.sub('^com[/:]','',ghome2)
                    m = m.split('/tree')[0]
                    if '/' not in m:
                        #print (objname, ghome)
                        pass
                except:
                    #print ('$$ Format Error', ghome)
                    pass
            if m in repodict.keys():
                repodict[m].append(objname)
            else:
                repodict[m] = [objname]
            
           
        
        

In [22]:
print(len(pkgs))
len(repodict)

3067


2646

In [18]:
repodict

{'naomik/bun': ['bun'],
 'substack/node-editor': ['editor'],
 '75lb/table-layout': ['table-layout'],
 'jfhbrook/node-ecstatic': ['ecstatic'],
 'sindresorhus/win-release': ['win-release'],
 '': ['@ionic/cli-framework'],
 'ember-cli/get-dependency-depth': ['ember-cli-get-dependency-depth'],
 'facebook/metro-bundler': ['metro-bundler'],
 'nfarina/xmldoc': ['xmldoc'],
 'rollup/rollup-plugin-commonjs': ['rollup-plugin-commonjs'],
 'ForbesLindesay/http-basic': ['http-basic'],
 'segmentio/superagent-retry': ['superagent-retry'],
 'novemberborn/release-zalgo': ['release-zalgo'],
 'wooorm/space-separated-tokens': ['space-separated-tokens'],
 'd3/d3-axis': ['d3-axis'],
 'atom/node-chromium-pickle-js': ['chromium-pickle-js'],
 'gruntjs/grunt-lib-phantomjs': ['grunt-lib-phantomjs'],
 'postcss/postcss-color-rgba-fallback': ['postcss-color-rgba-fallback'],
 'redux-utilities/reduce-reducers': ['reduce-reducers'],
 'dfilatov/node-inherit': ['inherit'],
 'sindresorhus/mimic-response': ['mimic-response'

In [23]:
for key in repodict.keys():
    if '.js' in key: 
        print(key, (repodict[key]))
        break

jeremyfa/yaml.js ['yamljs']


In [24]:
import json
with open('repolist_data_10k.json', 'w') as outfile:
    json.dump(repodict, outfile)